#2015 CAASPP test results

This notebook produces a CSV file that  compares the % MSA of San Diego county schools that have high disadvantage ( > 60% ) to the % of students that are disadvantaged. 

* [Source Website](http://caaspp.cde.ca.gov/sb2015/ResearchFileList)
* [Data Dictionary](http://caaspp.cde.ca.gov/sb2015/research_fixfileformat)

The county code for San Diego is 37.

The most important measurement values are: 
* pct_msa. The percentage of a subgroup that tested at or above the "Meets Standards" level. MSA = "Meet Standards or Above"
* mss. Mean Scale Score. The average test score for the subgroup. 

The geo_decomp() call seperates out the counties, schools and districts and limits the data to one grade. 

The first input cell restricts the data to the English Language Arts test. 


In [103]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import pandas as pd
df = pd.read_csv('../ca2015_all_csv_v1.txt', na_values='*'  )

# Entities. The school, district and county names. 
ent = pd.read_csv('../ca2015entities_csv.txt')

# Here is where we specify which test. 1 is the English / Language Arts test. 
#df = df[df['Test Id'] == 1]

# Create an identifier that combines the three codes. This ID is typical in other education files, 
# buyt oddly lacking from this one. 
def f(x):
    return "{:02d}{:05d}{:07d}".format(x['County Code'],x['District Code'],x['School Code'])
   
df['cds'] = df.apply(f, axis=1)
ent['cds'] = ent.apply(f, axis=1)
ent.set_index('cds',inplace = True) # index on cds so we can easily merge. 

county_names = ent['County Name']
school_names = ent['School Name']
district_names = ent[['District Name', 'County Name']]

# Friendly names of the subgroups. 
subgroups = pd.read_csv('../Subgroups.txt', header=None, names = ['subgroup_id','subgroup_name','category'],
                       index_col = 0)
subgroups['title'] = subgroups.apply(lambda row: row['category']+"; "+row['subgroup_name'], axis=1)
subgroup_names = subgroups.to_dict()['title']

# Calculate the percentage of students in the school, county or district who are members of the subgroup
df['pct_subgroup'] = df['Total Tested At Subgroup Level'] / df['Total Tested At Entity Level']

In [112]:
def col_subset(df):
    dfs = df[['cds', 'County Code', 'Subgroup ID', 'pct_subgroup', 'Grade', 'Test Id', 'Mean Scale Score', 
             'Percentage Standard Met and Above']]
    dfs.columns = [u'cds', 'county',  u'subgroup', u'pct_subgroup', u'grade', 'test', u'mss', u'pct_msa']
    return dfs


In [117]:
schools = col_subset(df[df['School Code'] != 0])
schools = schools[(schools.county == 37) & (schools.subgroup == 31)]
school_names = ent[['School Name','District Name', 'County Name']]
ed_rate = schools[(schools.subgroup == 31)][['cds', 'pct_subgroup']].copy()
ed_rate.columns = ['cds', 'pct_ec_disadvantaged']
ed_rate.pct_ec_disadvantaged = np.round(ed_rate.pct_ec_disadvantaged *100,0)
ed_rate = ed_rate.groupby('cds').max()

In [122]:
high_ed_x = schools[(schools.county==37) & (schools.subgroup == 31)].set_index('cds').join(school_names)\
.sort('pct_msa', ascending = False).join(ed_rate)

high_ed = high_ed_x[high_ed_x.pct_ec_disadvantaged > 60].sort('pct_msa', ascending = False).dropna()

In [125]:
high_ed.to_csv('high_ed.csv')